In [19]:
import pandas as pd
import time
import torch

## Загрузка данных

In [20]:
df = pd.read_csv('./data/data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [21]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [22]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [23]:
text[0]

['m',
 'a',
 'g',
 'g',
 'i',
 'e',
 ' ',
 'l',
 'o',
 'o',
 'k',
 ' ',
 'w',
 'h',
 'a',
 't',
 's',
 ' ',
 't',
 'h',
 'a',
 't']

## Делаем массив с данными то есть создаем наш словарь:

In [24]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [25]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)

for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [26]:
X[0:10]

tensor([[16, 12,  4,  4,  7, 18, 21, 27, 20, 20, 17, 21,  6,  3, 12, 22, 15, 21,
         22,  3, 12, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [27, 18, 18,  0, 16,  5,  9, 21, 27, 18, 18,  0, 16,  5,  9,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 8, 18, 18,  0, 24, 20, 20, 21,  8, 18, 18,  0, 24, 20, 20,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7, 16, 21, 22,  9, 10,  7, 25,  4, 21, 22, 20, 21, 22, 18, 12,  1,  3,
         21, 16, 12,  4,  4,  7, 18, 21, 22,  3, 12, 22, 21, 25, 12, 22,  5,  9,
         18, 21, 23, 20, 18, 15, 25, 22, 21, 18, 25, 23, 21,  6],
        [ 7, 22, 15, 21, 27,  7, 17, 18, 21, 12, 25, 21, 20, 13, 21, 20, 25, 27,
       

## Смотрим на Embedding и RNN ячейку

In [27]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:10])
t

tensor([[[-0.9061, -1.2353,  0.7974,  ...,  0.0033, -1.2187, -0.8302],
         [-0.4225, -0.0876, -0.1724,  ...,  0.6524,  0.4807, -2.0386],
         [-1.6586,  0.1539,  0.5801,  ...,  0.8103, -0.7022, -0.2910],
         ...,
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994],
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994],
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994]],

        [[-0.4540, -1.6186,  0.9846,  ..., -0.2465,  0.7290,  0.5281],
         [ 1.2531, -0.8410, -0.0323,  ...,  1.0981, -0.2238,  0.8668],
         [ 1.2531, -0.8410, -0.0323,  ...,  1.0981, -0.2238,  0.8668],
         ...,
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994],
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994],
         [ 1.9036,  0.9599,  0.2450,  ...,  1.2360,  0.0876, -0.1994]],

        [[-0.5864,  0.4137, -0.7974,  ...,  0.0323,  0.4236,  0.8530],
         [ 1.2531, -0.8410, -0.0323,  ...,  1

In [28]:
t.shape, X[0:10].shape

(torch.Size([10, 50, 28]), torch.Size([10, 50]))

In [29]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
# o - состоит из всех выходов скрытых состояний (10 предложений по 50 символов и 128 элементов )
# s - последнее скрытое состояние (1 скрытое состояние размерностью 128 )
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

In [14]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

## Практика. Реализуйте код модели нейронной сети
3 слоя - embeding (28), скрытая ячейка (128), полносвязанный из состояния rnn в букву (28)

In [30]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, a = self.rnn(embed)
        out = self.linear(o)
        return out

In [31]:
model = Network()

In [32]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [34]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 3.443, Train loss: 2.069
Epoch 1. Time: 2.728, Train loss: 1.822
Epoch 2. Time: 2.715, Train loss: 1.746
Epoch 3. Time: 2.601, Train loss: 1.699
Epoch 4. Time: 2.706, Train loss: 1.665
Epoch 5. Time: 2.686, Train loss: 1.639
Epoch 6. Time: 2.621, Train loss: 1.618
Epoch 7. Time: 2.688, Train loss: 1.599
Epoch 8. Time: 2.808, Train loss: 1.584
Epoch 9. Time: 2.888, Train loss: 1.569


## Практика. Реализуйте код генерации следующей буквы на основе модели
Логика такая:
    - Сначала кормим в нее буквы из sentence (прогревая состояние)
    - Затем пока не получим none (0) берем самую вероятную букву и добавляем ее в sentence
    - Повторяем

In [49]:
def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    # Todo
    x = torch.zeros((1, len(sentence)), dtype=int)

    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    for i in range(28):
        o = model(x)
        # print(o)
        # print(o.shape)
        w = torch.argmax(o[-1, -1, i], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0).unsqueeze(1)], axis=1)
        ww = INDEX_TO_CHAR[w]


In [50]:
generate_sentence()

In [52]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers= model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 3.214, Train loss: 1.557
Epoch 1. Time: 2.698, Train loss: 1.545
Epoch 2. Time: 2.746, Train loss: 1.534
Epoch 3. Time: 2.801, Train loss: 1.524
Epoch 4. Time: 3.000, Train loss: 1.514
Epoch 5. Time: 2.833, Train loss: 1.505
Epoch 6. Time: 2.776, Train loss: 1.496
Epoch 7. Time: 2.940, Train loss: 1.488
Epoch 8. Time: 2.677, Train loss: 1.480
Epoch 9. Time: 2.643, Train loss: 1.473
